In [1]:
import cv2
import numpy as np
import tensorflow as tf
from keras.models import model_from_json

C:\Users\Bhumit\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
# Load model
j_model = open('not-mnist-model.json', 'r')
model_json = j_model.read()
model = model_from_json(model_json)
j_model.close()
model.load_weights('not-mnist-weights.h5')

In [4]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [5]:
# Some helper functions
def x_cord_contours(contour):
    if cv2.contourArea(contour) > 10:
        M = cv2.moments(contour)
        return (int(M['m10'] / M['m00']))

    
def makeSquare(not_square):
    # This function takes an image and makes the dimenions square
    # It adds black pixels as the padding where needed
    
    BLACK = [0,0,0]
    img_dim = not_square.shape
    height = img_dim[0]
    width = img_dim[1]
    #print("Height = ", height, "Width = ", width)
    if (height == width):
        square = not_square
        return square
    else:
        doublesize = cv2.resize(not_square,(2*width, 2*height), interpolation = cv2.INTER_CUBIC)
        height = height * 2
        width = width * 2
        #print("New Height = ", height, "New Width = ", width)
        if (height > width):
            pad = (height - width)/2
            pad = int(pad)
            #print("Padding = ", pad)
            doublesize_square = cv2.copyMakeBorder(doublesize,0,0,pad,pad,cv2.BORDER_CONSTANT,value=BLACK)
        else:
            pad = (width - height)/2
            pad = int(pad)
            #print("Padding = ", pad)
            doublesize_square = cv2.copyMakeBorder(doublesize,pad,pad,0,0,cv2.BORDER_CONSTANT,value=BLACK)
    doublesize_square_dim = doublesize_square.shape
    #print("Sq Height = ", doublesize_square_dim[0], "Sq Width = ", doublesize_square_dim[1])
    return doublesize_square


def resize_to_pixel(dimensions, image):
    # This function then re-sizes an image to the specificied dimenions
    
    buffer_pix = 4
    dimensions  = dimensions - buffer_pix
    squared = image
    r = float(dimensions) / squared.shape[1]
    dim = (dimensions, int(squared.shape[0] * r))
    resized = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
    img_dim2 = resized.shape
    height_r = img_dim2[0]
    width_r = img_dim2[1]
    BLACK = [0,0,0]
    if (height_r > width_r):
        resized = cv2.copyMakeBorder(resized,0,0,0,1,cv2.BORDER_CONSTANT,value=BLACK)
    if (height_r < width_r):
        resized = cv2.copyMakeBorder(resized,1,0,0,0,cv2.BORDER_CONSTANT,value=BLACK)
    p = 2
    ReSizedImg = cv2.copyMakeBorder(resized,p,p,p,p,cv2.BORDER_CONSTANT,value=BLACK)
    img_dim = ReSizedImg.shape
    height = img_dim[0]
    width = img_dim[1]
    #print("Padded Height = ", height, "Width = ", width)
    return ReSizedImg

In [6]:
import cv2
def image_invert(image):
    #img = cv2.imread(image, 0)
    img = np.array(image)
    copy = np.copy(img)
    copy[copy>50] = 0
    copy[copy>0] = 255
    return copy

In [8]:
image = cv2.imread('test_letter.jpg')
gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
gray = cv2.resize(gray, (450,450))
cv2.imshow("image", image)
cv2.imshow("gray", gray)
cv2.waitKey(0)

# Blur image then find edges using Canny 
blurred = cv2.GaussianBlur(gray, (5, 5), 0)
cv2.imshow("blurred", blurred)
cv2.waitKey(0)


edged = cv2.Canny(blurred, 30, 150)
cv2.imshow("edged", edged)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Fint Contours
ret, contours, _ = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

#Sort out contours left to right by using their x cordinates
#contours = sorted(contours, key = x_cord_contours, reverse = False)
print(len(contours))

# Create empty array to store entire number
full_number = []

#load trained model
#model = load_mnist_model()

# loop over the contours
for c in contours:
    # compute the bounding box for the rectangle
    (x, y, w, h) = cv2.boundingRect(c)    
    
    #cv2.drawContours(image, contours, -1, (0,255,0), 3)
    #cv2.imshow("Contours", image)

    if w >= 35 and h >= 35:
        roi = gray[y:y + h, x:x + w]
        #ret, roi = cv2.threshold(roi, 150, 255,cv2.THRESH_BINARY_INV)
        roi = image_invert(roi)
        squared = makeSquare(roi)
        final = resize_to_pixel(28, squared)
        cv2.imshow("final", final)
        cv2.waitKey(0)
        final_array = final.reshape((1,28,28,1))
        final_array = final_array.astype(np.float32)
        #ret, result, neighbours, dist = knn.find_nearest(final_array, k=1)
        #number = str(int(float(result[0])))
        number = model.predict_classes(final_array)[0]
        full_number.append(number)
        # draw a rectangle around the digit, the show what the
        # digit was classified as
        #cv2.rectangle(image, (x, y), (x + w, y + h), (0, 0, 255), 2)
        #cv2.putText(image, number, (x , y + 155),
        #    cv2.FONT_HERSHEY_COMPLEX, 2, (255, 0, 0), 2)
        #cv2.imshow("image", image)
        #cv2.waitKey(0) 
        
cv2.destroyAllWindows()
print('Number is')
print(full_number)

5
Number is
[0, 1, 2, 3, 4]
